# Simulation : Robots vs Dinosaurs


## Motivation
The primary purpose in this challenge is to create simulation in a __feature space__ for agents to act. For completing this challenge Python 3.0 is used along with the simulation class function.

In [1]:
# import libraries
import numpy as np
from flask import Flask, request
from flask_restful import Resource, Api

## Methodology
The `methodology` followed here involves the following:

1. Creating a simulation space of matrix where arbitrarily position is marked with coordinates.
2. We set or mark the robot with a 1.0 label and dino with a 2.0 label.
3. The class function contains modules where we call these methods to instruct in simulation.
4. For each module we have created test cases and edge test cases.
5. We have converted this simulation to Rest API using Flask
6. The code is published in Github in this [link](https://github.com/SaurabhChakravorty/Data-Science-Projects/upload/master/Interview_Case_Studies) by the problem statement present [here](https://github.com/devsbb/grover-engineering-recruitment/blob/master/challenges/robots-vs-dinos/ASSIGNMENT.md).

In [2]:
class simulation:
    '''
    This class contains all the modules or methods for the simulation process
    '''
    
    def __init__(self, sample_size = 50):
        # create an numpy array of fixed size
        self.env = np.zeros((sample_size, sample_size))
        
        # intitialising the position of robot and dino
        self.robot = (0,0)
        self.env[self.robot] = 1.0 
        
        self.dino  = (1,1)
        self.env[self.dino] = 2.0 
        
        # for storing the location of "kill" of robot
        self.kill = []
        
    def agent_position(self, agent = "robot", position = (0,0)):     
        # changing the state based on condition
        if agent == "robot":
            env = self.robot
            mark = 1.0         
        else:
            env = self.dino
            mark = 2.0
            
        # assigning the position to agent 
        if self.__check_position__(position) and self.env[position] == 0.0 :
            self.env[env] = 0.0                      # setting the initial pos to 0.0
            if agent == "robot":
                self.robot = position
            else:
                self.dino = position
            self.env[position] = mark                # assigning Robot pos as 1.0 if condition is met          
        else:
            print("The position of {} cannot be updated in feature space".format(agent))    
            
    def move_robot(self, direction = "left"):
         # for moving the 'Robot' agent in simulation space
         if direction == 'left':
            self.robot = (self.robot[0] - 1, self.robot[1])
            
         if direction == 'right':
            self.robot = (self.robot[0] + 1, self.robot[1])
            
         if direction == 'top':
            self.robot = (self.robot[0], self.robot[1] + 1)
                          
         if direction == 'bottom':
            self.robot = (self.robot[0], self.robot[1] - 1)
       
         # check position and store the space
         self.agent_position(agent = "robot", position = self.robot)
         self.__display_position__()
            
    def attack_dino(self): 
        # attack in vicinity
        if np.abs(self.robot[0] - self.dino[0]) == 1 or np.abs(self.robot[1] - self.dino[1]) == 1 :
            self.robot = self.dino          # update the position
            self.env[self.robot] = 1.0     

            self.kill.append(self.robot)    # position where the kill happened
            self.dino = (0,0)               # initial pos again
        
            print("The dinosaur is killed at position {} and position of dinosaur is set to (0,0)".format(self.robot))
        else:
            print("We can't kill the dinosaur as it is far...")
                    
                    
    def __display_position__(self):
        # for displaying the positions of Dino and Robot
        print("The position of Robot is {} and position of Dinosaur is {}".format(self.robot, self.dino))
    
    
    def __check_position__(self, pos):
        # for checking the position in feature space
        if (0 <= pos[0] < self.env.shape[0]) and (0 <= pos[1] < self.env.shape[1]):
            return True
        else:
            return False

## Simulation Test cases

### Simple Test Cases

In [3]:
# empty simulation space
s = simulation(sample_size=50)

# mark 'Robot' position
s.agent_position(agent = "robot", position = (14, 2))

# mark 'Dinosaur' position
s.agent_position(agent = "dinosaur", position = (15, 2))

In [4]:
# check if robot is moving
s.move_robot(direction = "top")

The position of Robot is (14, 3) and position of Dinosaur is (15, 2)


In [5]:
# kill the "dino"
s.attack_dino()

The dinosaur is killed at position (15, 2) and position of dinosaur is set to (0,0)


In [6]:
# display the current state
s.__display_position__()

The position of Robot is (15, 2) and position of Dinosaur is (0, 0)


### Edge Cases

In [7]:
# check whether we can put the "robot" and "dinosaur" in same state
# mark 'Robot' position
s.agent_position(agent = "robot", position = (25, 25))

# mark 'Dinosaur' position
s.agent_position(agent = "dinosaur", position = (25, 25))

The position of dinosaur cannot be updated in feature space


> We see as the "Robot" is in that poisition previously we cant update it simultaneously

In [8]:
# check whether we can put the "robot" and "dinosaur" in same state
# mark 'Robot' position
s.agent_position(agent = "robot", position = (25, 50))

# mark 'Dinosaur' position
s.agent_position(agent = "dinosaur", position = (15, 25))

The position of robot cannot be updated in feature space


In [9]:
# kill the "dino"
s.attack_dino()

We can't kill the dinosaur as it is far...


## Implementation using REST API

We will be using __Flask__ for REST API implementation. The request will go in the form of http POST request. 

In [10]:
from flask import Flask, request
from flask_restful import Resource, Api
from main_code import simulation

In [11]:
# initialise the flask
app = Flask(__name__)
api = Api(app)

class test_cases(Resource):
    def __init__(self, sample_size = 50, robot_pos = (14, 2) , dino_pos = (15, 2) , direction = "top"):
        # make it class variables
        self.sample_size = sample_size
        self.robot_pos   = robot_pos
        self.dino_pos    = dino_pos
        self.direction   = direction
    
    def get(self):
        # empty simulation space
        s = simulation(sample_size=self.sample_size)

        # mark 'Robot' position
        s.agent_position(agent = "robot", position = self.robot_pos)

        # mark 'Dinosaur' position
        s.agent_position(agent = "dinosaur", position = self.dino_pos)
        
        # check if robot is moving
        s.move_robot(direction = self.direction)
        
        # kill the "dino"
        s.attack_dino()
        
        # display the current state
        s.__display_position__()
              
api.add_resource(test_cases, '/')

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with windowsapi reloader


SystemExit: 1

C:\Users\Saurabh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
